<a href="https://colab.research.google.com/github/bodorcy/hazifeladatok/blob/main/ml_5_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning
Olvasd el az [elméleti bevezetőt](http://inf.u-szeged.hu/~rfarkas/deep_learning.html).

### Futtatás GPU-n

A mély neurális hálók tanítása nagyon számításigényes, viszont visszavezetve mátrixműveletekre nagyon jól párhuzamosítható GPU-n. Érdemes a Google Colab-ban is átváltani GPU-ra. Ezt az Edit>Notebook settings menüben tehetjük meg GPU-t választva hardveres gyorsításra. Ha CPU-ról átvátunk GPU-ra akkor újra kell futtatni a teljes notebookot!

A Cuda egy alacsony szintű szoftverréteg mátrixműveletek GPU-n való nagyon hatékony megvalósítására. E fölé épülnek a deep learning keretrendszerek, pl.  [PyTorch](https://pytorch.org/) és a [Tensorflow](https://www.tensorflow.org/).

In [ ]:
### PyTorch deep learning keretrendszert használjuk: https://pytorch.org
import torch

In [ ]:
### Futtatási környezet előkészítése

# Cuda inicializálása
torch.backends.cudnn.deterministic = True

# a neurális hálók tanításánál a véletlenszám-generálásnak nagy szerepe van
# érdemes a random seedet fixálni, hogy minden futtatásra ugyanazt az eredményt kapjuk
SEED = 202004
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Szövegosztályozás mély tanulással

Az [elöző](https://colab.research.google.com/drive/1Ve2FOeA7ceEgS0eqL-31CUFFT33s_PDm) órán megoldott szövegosztályozási feladatra fogunk adni itt egy mély gépi tanulási megoldást. Ugyanaz a feladat, véleményosztályozás. Ugyanazon az adatbázison, ugyanazon kiértékelési metrikát használjuk, így az eredmények összehasonlíthatóak a klasszikus gépi tanulási eredményekkel.

In [ ]:
import pandas as pd
train_data = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/sentiment/train.tsv', sep='\t')
test_data  = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/sentiment/test.tsv', sep='\t')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer()
cv_counts = vectorizer.fit_transform(train_data.text)
idf_transformer = TfidfTransformer(use_idf=True).fit(cv_counts)
features = idf_transformer.transform(cv_counts)
test_features = idf_transformer.transform(vectorizer.transform(test_data.text))

In [ ]:
features

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 147011 stored elements and shape (9063, 24285)>

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
model = SGDClassifier().fit(features, train_data.label)
accuracy_score(y_true=test_data.label, y_pred=model.predict(test_features))

0.7893333333333333

## Egyszerű neurális hálózat

In [ ]:
### ritka mátrixot tensor formátumra alakítjuk
import numpy as np
X_train_tensor = torch.from_numpy(features.todense()).float()
X_test_tensor  = torch.from_numpy(test_features.todense()).float()

In [ ]:
train_data.label

,label
0,NEGATIVE
1,NEUTRAL
2,POSITIVE
3,NEGATIVE
4,NEGATIVE
...,...
9058,NEUTRAL
9059,NEUTRAL
9060,POSITIVE
9061,NEGATIVE


In [ ]:
### PyTorch-ban még a célváltozó sem lehet diszkrét...
### A LabelEncoder véletlenszerűen Int-eket rendel az egyes értékekhez
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train_tensor = torch.as_tensor(le.fit_transform(train_data.label))
Y_test_tensor  = torch.as_tensor(le.transform(test_data.label))

In [ ]:
### Jellemzőtér (=bemeneti réteg) dimenziói és célváltozók száma (=kimeneti réteg dimenziója)
VOCAB_SIZE = len(vectorizer.vocabulary_)
OUT_CLASSES = 3

In [ ]:
### Linear Machine, LM
### A legegyszerűbb neurális háló (ami megegyezik a lineáris géppel)
### a kimeneti neuron össze vannak kötve a bementiekkel (mindegyik mindegyikkel)

class LM_Network(torch.nn.Module):
     def __init__(self,vocab_size,out_classes):
        super().__init__()
        self.linear = torch.nn.Linear(vocab_size,out_classes)
     def forward(self,x):
        return self.linear(x)

model = LM_Network(VOCAB_SIZE,OUT_CLASSES)
print(model)

LM_Network(
  (linear): Linear(in_features=24285, out_features=3, bias=True)
)


In [ ]:
# összesen ennyi paramétert kell tanítanunk:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(model)

72858

In [ ]:
#predikció a random hálóval
model(X_train_tensor[1])

tensor([-0.0019, -0.0008,  0.0132], grad_fn=<ViewBackward0>)

In [ ]:
### Multi Layer Perceptron, MLP
### 1 rejtett réteget tartalmazó neuárlis hálózat

class MLP_Network(torch.nn.Module):
  def __init__(self,vocab_size,hidden_units,num_classes):
      super().__init__()
      #First fully connected layer
      self.fc1 = torch.nn.Linear(vocab_size,hidden_units)
      #Second fully connected layer
      self.fc2 = torch.nn.Linear(hidden_units,num_classes)
      #Final output of sigmoid function
      self.sigmoid = torch.nn.Sigmoid()

  def forward(self,x):
      y1 = self.sigmoid(self.fc1(x))
      output = self.sigmoid(self.fc2(y1))
      return output

HIDDEN_UNITS = 100
model = MLP_Network(VOCAB_SIZE, HIDDEN_UNITS, OUT_CLASSES)
print(model)
print(count_parameters(model), "tanulandó paraméter")

MLP_Network(
  (fc1): Linear(in_features=24285, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=3, bias=True)
  (sigmoid): Sigmoid()
)
2428903 tanulandó paraméter


In [ ]:
### Kiértékelő függvény
def accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True) # a 3 osztályra adott kimeneti érték közül melyik a legnagyobb
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum(dtype=float) / y.shape[0]

In [ ]:
### ha az epoch végén egy független validációs halmazon is ki akarjuk értékelni a modellt:
def evaluate(model, iterator):
    epoch_acc = 0
    model.eval()  # inicializálás
    with torch.no_grad():
        for batch in iterator:
            # predikció
            predictions = model(batch[0])
            # kiértékelés
            acc = accuracy(predictions, batch[1].long())
            epoch_acc += acc.item()

    return epoch_acc / len(iterator)

In [ ]:
Y_test_tensor

In [ ]:
from torch.utils.data import Dataset, TensorDataset
train_data = TensorDataset(X_train_tensor, Y_train_tensor)
test_data  = TensorDataset(X_test_tensor,  Y_test_tensor)

In [ ]:
### Ha egy adatbázison akarunk végigmenni akkor ahhoz iterátort kell definiálni
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

In [ ]:
# random hálózat kiértékelése az egész adatbázison
evaluate(model, train_loader)

0.9106827916351726

In [ ]:
### A tanítás során többször végigmegyünk a tanító adatbázison (egy kör egy epoch)
def train(model, iterator, optimizer, criterion):
    # minden epoch végén ellenőrízni fogjuk az accuracyt
    epoch_acc = 0

    model.train() # inicializálás
    for batch in iterator:
        # predikáljuk le a tanító példákat az aktuális paraméterekkel:
        optimizer.zero_grad()
        predictions = model(batch[0])

        # a háló aktuális paraméterivel ennyi a hiba a batchen:
        loss = criterion(predictions, batch[1].long())
        acc = accuracy(predictions, batch[1].long())

        # hibavisszaterjesztéssel (backpropagation) javítunk a paramétereken:
        loss.backward()
        optimizer.step()

        epoch_acc += acc.item()

    return epoch_acc / len(iterator)

In [ ]:
### Neurális hálózat tanítása
%%time
NUM_EPOCHS = 10
BATCH_SIZE = 64

#Neurális háló architektúra megadása
model = MLP_Network(VOCAB_SIZE,HIDDEN_UNITS,OUT_CLASSES)

#optimalizáló eljárás
import torch.optim as optim
optimizer = optim.Adam(model.parameters()) # ADAM optimalizáló algoritmus

#célfüggvény
import torch.nn as nn
loss_fun = nn.CrossEntropyLoss()

iterator = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True)
for i in range(NUM_EPOCHS):
   print(i, ". epoch acc:", train(model, iterator, optimizer, loss_fun))

0 . epoch acc: 0.37931112314915133
1 . epoch acc: 0.6118127031419285
2 . epoch acc: 0.7679216323582521
3 . epoch acc: 0.847979302094619
4 . epoch acc: 0.8713773022751896
5 . epoch acc: 0.8951844077284218
6 . epoch acc: 0.9079400279884435
7 . epoch acc: 0.9194937251715422
8 . epoch acc: 0.9302856401227879
9 . epoch acc: 0.9384677455760202
CPU times: user 36.8 s, sys: 9.25 s, total: 46 s
Wall time: 46.3 s


In [ ]:
### Kiértékelés a teszt halmazon
test_loader = DataLoader(test_data,batch_size=16, shuffle=True)
evaluate(model, test_loader)

0.7732712765957447

In [ ]:
### Futtassunk mindent GPU-n!
### Mindent át kell pakolni a GPU memóriájába...

%%time
NUM_EPOCHS = 10
BATCH_SIZE = 64

#Initialize model
model = MLP_Network(VOCAB_SIZE,HIDDEN_UNITS,OUT_CLASSES).to(device)

#Initialize optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters()) # ADAM optimalizáló algoritmus
import torch.nn as nn
loss_fun = nn.CrossEntropyLoss().to(device)

X_train_tensor = X_train_tensor.to(device)
Y_train_tensor = Y_train_tensor.to(device)
train_data = TensorDataset(X_train_tensor, Y_train_tensor)
iterator = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True)

for i in range(NUM_EPOCHS):
   print(i, ". epoch acc:", train(model, iterator, optimizer, loss_fun))

0 . epoch acc: 0.4128013271939328
1 . epoch acc: 0.6051880191404839
2 . epoch acc: 0.7678595612134345
3 . epoch acc: 0.849528259299386
4 . epoch acc: 0.8728867596605273
5 . epoch acc: 0.894154591007584
6 . epoch acc: 0.9066591052726617
7 . epoch acc: 0.9193131545684363
8 . epoch acc: 0.9303561755146263
9 . epoch acc: 0.9395201336222463
CPU times: user 3.65 s, sys: 252 ms, total: 3.9 s
Wall time: 4.58 s


In [ ]:
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)
test_data = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_data,batch_size=16, shuffle=True)
evaluate(model, test_loader)

# Konvolúciós Neurális Hálózatok (CNN)

Egy ún **Konvulúciós Neurális Hálózatot** fogunk építani és tanítani a szövegosztályozási feladathoz (lásd [olvasólecke](https://www.inf.u-szeged.hu/~rfarkas/ML22/deep_learning.html)).

## Idősor Előrejelzés

A konvolúciós hálók, amik mozgó ablakos szűrők segítségével tömörítik az információt alkalmasak képek, szövegek és idősorok feldolgozására is.

A következő szakaszban idősorok előrejelzésére az American Electric Power áramszolgáltató fogyasztási adatait fogjuk használni. A cél, hogy meg tudjuk mondani a jövőben mekkora lesz a fogyasztás. Mivel egy folytonos értéket akarunk megbecsülni, ezért ez egy regressziós feladat lesz.

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
file_path = "AEP_hourly.csv"

# használandó adathalmaz, áramfigyasztások óránként amerikában
# https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "robikscube/hourly-energy-consumption",
  file_path,
)

In [ ]:
df.Datetime = pd.to_datetime(df.Datetime)
df = df.set_index('Datetime')

In [ ]:
df.AEP_MW.plot()

In [ ]:
df.shape

### Tanító adathalmaz készítése

Az idősorból két napnyi szakaszokat szedünk ki, és ezek alapján a szakaszok alapján próbáljuk megbecsülni a követkető időpont áramfogyasztását.

In [ ]:
window_size = 48
#import numpy as np
# train halmaz
features = []
labels = []
for i in range(window_size, len(df.AEP_MW)):
    features.append([df.AEP_MW.iloc[i - window_size:i].values])
    labels.append(df.AEP_MW.iloc[i])

chunk_df = pd.DataFrame({"features": features, "labels": labels})

In [ ]:
chunk_df.features[0]

A chunkok alapján elkészítjük a tanító és a kiértékelő adathalmazt.

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch import Tensor
import torch.nn.functional as F
import torch


def collate_batch(batch):
    label_list, text_list = [], []
    for row in batch:
        # Jellemzők átalakítása
        text_list.append(Tensor(row["features"]))

        # Címkék átalakítása
        label_list.append(row["labels"])

    labels_tensor = Tensor(label_list).to(device)

    feature_vec_tensor = Tensor(np.array(text_list)).to(device)

    return labels_tensor, feature_vec_tensor


In [ ]:
# basic train/test split
split_point = int(len(df) * 0.8)
train_df = chunk_df[:split_point]
test_df = chunk_df[split_point:]

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,
                          collate_fn=collate_batch)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
                          collate_fn=collate_batch)

In [ ]:
print(x := next(iter(train_loader)))
print(Tensor(x[1]).shape)
print(Tensor(x[0]).shape)

### Baseline

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor

# baseline regressor
dummy_reg = DummyRegressor(strategy="mean")
dummy_reg.fit(train_df.features, train_df.labels)
pred_y = dummy_reg.predict(test_df.features)

mean_absolute_error(test_df.labels, pred_y)

### CNN szerkezetének megadása

Minden feladatra saját hálózatot építhetünk az egyes neuron rétegek megadásával. Ehhez egy új osztályt kell definiálni, legalább konstruktorral és forward() metódussal.

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_channels, n_filters, kernel_size, output_dim):
        super().__init__()

        # konvolúciós réteg
        self.conv = nn.Conv1d(
            in_channels = input_channels,  # mennyi csatornát adunk be: pl. 1, ha egyváltozós a sor, vagy >1, ha több feature/ablak
            out_channels = n_filters,     # hány különböző konvolúciós szűrőt alkalmazunk
            kernel_size = kernel_size     # "ablakméret": hány időlépést nézünk egyszerre
        )

        # kimeneti réteg, ami egy egyszerű lineáris réteg
        self.fc = nn.Linear(n_filters, output_dim)

    def forward(self, x):
        # x: [batch_size, csatornák, seq_len]

        # Konvolúciós réteg + ReLU aktiváció
        conved = F.relu(self.conv(x))
        # conved = [batch_size, n_filters, seq_len - kernel_size + 1]

        # tovább tömörítjük:
        pooled = F.max_pool1d(conved, conved.shape[2]).squeeze(2)
        # print("pooled", pooled.size())
        # pooled = [batch_size, n_filters]

        # a háló kimenetét a lineáris réteg számolja ki
        return self.fc(pooled)

In [ ]:
#---------------<params>---------------

input_channels = 1
n_filters = 40
kernel_size = 12
output_dim = 1 # mivel regresszióról beszélünk csak 1 neuronra van szükgésünk kiementként

#--------------------------------------

# modell példányosítása
model = CNN(input_channels=input_channels, n_filters=n_filters, kernel_size=kernel_size, output_dim=output_dim)

# ha lehet használjunk gpu-t
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# a háló rétegei:
print(model)

# összesen ennyi paramétert kell tanítanunk:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(model), "tanulandó változó")

### CNN tanítása

A neurális hálók tanítása egy optimalizációs feladat megoldásával törénik. Úgy akrjuk beállítani a háló súlyait, hogy minimalizáljuk a háló kimenet és a tényleges megjóslandó érték közti eltérést.

A hiba számítására az átlagos négyzetes hibát (Mean Squared Error, MSE) fogjuk használni.

In [ ]:
# optimizer és hibafüggvény
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [ ]:
### A tanítás során többször végigmegyünk a tanító adatbázison (egy kör egy epoch)

def cnn_train(model, iterator, optimizer, criterion):
    model.train()
    total_loss = 0
    all_preds = []
    all_y = []
    for batch in train_loader:
        X = batch[1].to(device)
        y = batch[0].to(device)

        optimizer.zero_grad()
        preds = model(X).squeeze(1)

        all_preds.extend(preds.tolist())
        all_y.extend(y.tolist())

        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader), all_y, all_preds

def cnn_eval(preds, test_loader):
    model.eval()
    total_loss = 0

    all_preds = []
    all_y = []

    with torch.no_grad():
        for batch in test_loader:
            X = batch[1].to(device)
            y = batch[0].to(device)
            preds = model(X).squeeze(1)

            all_preds.extend(preds.tolist())
            all_y.extend(y.tolist())

            loss = criterion(preds, y)
            total_loss += loss.item()

    return total_loss / len(test_loader), all_y, all_preds

In [ ]:
%%time
### mehet a tanítás!

NUM_EPOCHS = 10

for i in range(NUM_EPOCHS):
    train_loss, train_labels, train_preds = cnn_train(model, train_loader, optimizer, criterion)
    test_loss, test_labels, test_preds = cnn_eval(model, test_loader)

    train_mae = mean_absolute_error(train_labels, train_preds)
    test_mae = mean_absolute_error(test_labels, test_preds)

    print(i, ". epoch train MAE:", train_mae, " test MAE:", test_mae)

Predikciók vizsgálata.

In [ ]:
pd.DataFrame({"test_labels": train_labels, "test_preds": train_preds})[:100].plot()